# Remote actions and variables

Changing the state of remote resources


---

- Website: https://discovery.gitlabpages.inria.fr/enoslib/index.html
- Instant chat: https://framateam.org/enoslib
- Source code: https://gitlab.inria.fr/discovery/enoslib

---




**Prerequisites**:
- A Grid'5000 account
- A working EnOSlib environment and Jupyter (not included in EnOSlib dependencies, but `pip install jupyterlab` will install it)




In [1]:
import enoslib as en

# get some logging info
import logging
logging.basicConfig(level=logging.INFO)

## Setup on Grid'5000

EnOSlib uses `Providers` to ... provide resources. They transform an abstract resource configuration into a concrete one.
To do so, they interact with an infrastructure where they get the resources from. There are different providers in EnOSlib: 

- Vbox/KVM to work with locally hosted virtual machines
- Openstack/Chameleon to work with bare-metal resources hosted in the Chameleon platform
- FiT/IOT lab to work with sensors or low profile machines
- VmonG5k to work with virtual machines on Grid'5000
- Distem to work with lxc containers on Grid'5000
- **Grid'5000**, with options to configure several networks easily

A providers eases the use of the platform by internalizing some of the configuration tasks (e.g automatically managing the reservation on G5k, network configuration ...)

### Describing the resources

For the purpose of the tutorial we'll reserve 2 nodes in the production environment.

First we build a configuration object describing the wanted resources: `machines` and `networks`.

In [2]:
network = en.G5kNetworkConf(type="prod", roles=["my_network"], site="rennes")

conf = (
    en.G5kConf.from_settings(job_type="allow_classic_ssh", job_name="rsd-01")
    .add_network_conf(network)
    .add_machine(
        roles=["control"], cluster="parasilo", nodes=1, primary_network=network
    )
    .add_machine(
        roles=["compute"],
        cluster="parasilo",
        nodes=1,
        primary_network=network,
    )
    .finalize()
)
conf

Conf@0x7f27c48583d0
{
    "dhcp": true,
    "force_deploy": false,
    "env_name": "debian10-x64-nfs",
    "job_name": "rsd-01",
    "job_type": "allow_classic_ssh",
    "key": "/home/msimonin/.ssh/id_rsa.pub",
    "queue": "default",
    "walltime": "02:00:00",
    "resources": {
        "machines": [
            {
                "roles": [
                    "control"
                ],
                "primary_network": "6cbb1585-cc96-47aa-8fa2-d03595f7ac01",
                "secondary_networks": [],
                "cluster": "parasilo",
                "nodes": 1
            },
            {
                "roles": [
                    "compute"
                ],
                "primary_network": "6cbb1585-cc96-47aa-8fa2-d03595f7ac01",
                "secondary_networks": [],
                "cluster": "parasilo",
                "nodes": 1
            }
        ],
        "networks": [
            {
                "id": "6cbb1585-cc96-47aa-8fa2-d03595f7ac01",
                "type": "prod",
                "roles": [
                    "my_network"
                ],
                "site": "rennes"
            }
        ]
    }
}

### Reserving the resources

We can pass the `Configuration` object to the `G5k` provider. 

In [3]:
provider = en.G5k(conf)
roles, networks = provider.init()

INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from grenoble
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from lille
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from luxembourg
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from lyon
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from nancy
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from nantes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from rennes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from sophia
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Submitting {'name': 'rsd-01', 'types': ['allow_classic_ssh'], 'resources': "{cluster='parasilo'}/nodes=1+{cluster='parasilo'}/nodes=1,walltime=02:00:00", 'command': 'sleep 31536000', 'queue': 'default'} on rennes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Waiting for 1820191 on rennes [2021-09-27 10:23:23]
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Waiting for 1820191 on rennes [2021-09-

Inspecting the ressources we own for the experiment's lifetime:

- roles: this is somehow a dictionnary whose keys are the role names and the associated values are the corresponding list of hosts
- networks: similar to roles bu£t for networks

In [4]:
roles

{'control': [Host(address='parasilo-23.rennes.grid5000.fr', alias='parasilo-23.rennes.grid5000.fr', user='root', keyfile=None, port=None, extra={}, net_devices=set())], 'compute': [Host(address='parasilo-8.rennes.grid5000.fr', alias='parasilo-8.rennes.grid5000.fr', user='root', keyfile=None, port=None, extra={}, net_devices=set())]}

In [5]:
# list of host on a given role
roles["control"]

[Host(address='parasilo-23.rennes.grid5000.fr', alias='parasilo-23.rennes.grid5000.fr', user='root', keyfile=None, port=None, extra={}, net_devices=set())]

In [6]:
# a single host
roles["control"][0]

Host(address='parasilo-23.rennes.grid5000.fr', alias='parasilo-23.rennes.grid5000.fr', user='root', keyfile=None, port=None, extra={}, net_devices=set())

In [7]:
networks

{'my_network': [<enoslib.infra.enos_g5k.objects.G5kEnosProd4Network object at 0x7f27c48587d0>, <enoslib.infra.enos_g5k.objects.G5kEnosProd6Network object at 0x7f27c3e43cd0>]}

`provider.init` is idempotent. In the Grid'5000 case, you can call it several time in a row. The same reservation will reloaded and the roles and networks will be the same.

In [8]:
roles, networks = provider.init()
roles

INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from grenoble
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from lille
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from luxembourg
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from lyon
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from nancy
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from nantes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from rennes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading 1820191 from rennes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from sophia
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Waiting for 1820191 on rennes [2021-09-27 10:23:23]
INFO:enoslib.infra.enos_g5k.g5k_api_utils:All jobs are Running !


{'control': [Host(address='parasilo-23.rennes.grid5000.fr', alias='parasilo-23.rennes.grid5000.fr', user='root', keyfile=None, port=None, extra={}, net_devices=set())], 'compute': [Host(address='parasilo-8.rennes.grid5000.fr', alias='parasilo-8.rennes.grid5000.fr', user='root', keyfile=None, port=None, extra={}, net_devices=set())]}

In [9]:
# sync some more information in the host data structure (for illustration purpose here)
roles = en.sync_info(roles, networks)

[WARNING]: No inventory was parsed, only implicit localhost is available



PLAY [all] **********************************************************************************************************************************************************************************

TASK [hostname] *****************************************************************************************************************************************************************************
 [started TASK: hostname on parasilo-23.rennes.grid5000.fr]
 [started TASK: hostname on parasilo-8.rennes.grid5000.fr]
changed: [parasilo-23.rennes.grid5000.fr]
changed: [parasilo-8.rennes.grid5000.fr]

PLAY [Gather facts for all hosts] ***********************************************************************************************************************************************************

TASK [Gathering Facts] **********************************************************************************************************************************************************************
ok: [parasilo-23.rennes.grid5000.f

In [10]:
# the hosts have been populated with some new information
roles

ip
::1/128
127.0.0.1/8
ip
fe80::eef4:bbff:fed1:e60/64
172.16.97.23/20
ip
::1/128
127.0.0.1/8
ip
fe80::eef4:bbff:fed0:f248/64


## Acting on remote nodes

### run a command, filter results

In [11]:
results = en.run_command("whoami", roles=roles)
results


PLAY [all] **********************************************************************************************************************************************************************************

TASK [whoami] *******************************************************************************************************************************************************************************
 [started TASK: whoami on parasilo-23.rennes.grid5000.fr]
 [started TASK: whoami on parasilo-8.rennes.grid5000.fr]
changed: [parasilo-8.rennes.grid5000.fr]
changed: [parasilo-23.rennes.grid5000.fr]


CommandResult(host='parasilo-8.rennes.grid5000.fr', task='whoami', status='OK', payload={'cmd': 'whoami', 'stdout': 'root', 'stderr': '', 'rc': 0, 'start': '2021-09-27 10:23:53.423560', 'end': '2021-09-27 10:23:53.432343', 'delta': '0:00:00.008783', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'whoami', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['root'], 'stderr_lines': [], '_ansible_no_log': False})
CommandResult(host='parasilo-23.rennes.grid5000.fr', task='whoami', status='OK', payload={'cmd': 'whoami', 'stdout': 'root', 'stderr': '', 'rc': 0, 'start': '2021-09-27 10:23:53.427285', 'end': '2021-09-27 10:23:53.436010', 'delta': '0:00:00.008725', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'whoami', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['root'], 'stderr_lines': [], '_ansible_no_log': False})

In [12]:
one_result = results.filter(host=roles["control"][0].alias)[0]
one_result

CommandResult(host='parasilo-23.rennes.grid5000.fr', task='whoami', status='OK', payload={'cmd': 'whoami', 'stdout': 'root', 'stderr': '', 'rc': 0, 'start': '2021-09-27 10:23:53.427285', 'end': '2021-09-27 10:23:53.436010', 'delta': '0:00:00.008725', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'whoami', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['root'], 'stderr_lines': [], '_ansible_no_log': False})

In [13]:
one_result.payload["stdout"]

'root'

There are some specific shortcuts when the remote actions is a remote (shell) command: `.stdout`, `.stderr`, `.rc`

In [14]:
print(f"stdout = {one_result.stdout}\n", f"stderr={one_result.stderr}\n", f"return code = {one_result.rc}")

stdout = root
 stderr=
 return code = 0


By default the user is `root` (this is common to all EnOSlib's provider).
If you want to run command as your regular Grid'5000 user you can tell the command to `sudo` back to your regular user using `run_as` (the SSH login is still `root` though)

In [15]:
my_g5k_login = en.g5k_api_utils.get_api_username()
results = en.run_command("whoami", roles=roles, run_as=my_g5k_login)
results


PLAY [all] **********************************************************************************************************************************************************************************

TASK [whoami] *******************************************************************************************************************************************************************************
 [started TASK: whoami on parasilo-23.rennes.grid5000.fr]
 [started TASK: whoami on parasilo-8.rennes.grid5000.fr]
changed: [parasilo-23.rennes.grid5000.fr]
changed: [parasilo-8.rennes.grid5000.fr]


CommandResult(host='parasilo-23.rennes.grid5000.fr', task='whoami', status='OK', payload={'cmd': 'whoami', 'stdout': 'msimonin', 'stderr': '', 'rc': 0, 'start': '2021-09-27 10:23:53.902284', 'end': '2021-09-27 10:23:53.904619', 'delta': '0:00:00.002335', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'whoami', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['msimonin'], 'stderr_lines': [], '_ansible_no_log': False})
CommandResult(host='parasilo-8.rennes.grid5000.fr', task='whoami', status='OK', payload={'cmd': 'whoami', 'stdout': 'msimonin', 'stderr': '', 'rc': 0, 'start': '2021-09-27 10:23:53.902685', 'end': '2021-09-27 10:23:53.904988', 'delta': '0:00:00.002303', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'whoami', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['msimonin'], 'stderr_lines': [], '_ansible_no_log': False})

### Filtering hosts on which the command is run

`run_command` acts on remote hosts. Those hosts can be given as a `Roles` type (output of `provider.init`) or as a list of `Host` or a single `Host`. 


In [16]:
# some roles
en.run_command("date", roles = roles)


PLAY [all] **********************************************************************************************************************************************************************************

TASK [date] *********************************************************************************************************************************************************************************
 [started TASK: date on parasilo-23.rennes.grid5000.fr]
 [started TASK: date on parasilo-8.rennes.grid5000.fr]
changed: [parasilo-23.rennes.grid5000.fr]
changed: [parasilo-8.rennes.grid5000.fr]


CommandResult(host='parasilo-23.rennes.grid5000.fr', task='date', status='OK', payload={'cmd': 'date', 'stdout': 'Mon 27 Sep 2021 10:23:54 AM CEST', 'stderr': '', 'rc': 0, 'start': '2021-09-27 10:23:54.264575', 'end': '2021-09-27 10:23:54.266866', 'delta': '0:00:00.002291', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'date', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['Mon 27 Sep 2021 10:23:54 AM CEST'], 'stderr_lines': [], '_ansible_no_log': False})
CommandResult(host='parasilo-8.rennes.grid5000.fr', task='date', status='OK', payload={'cmd': 'date', 'stdout': 'Mon 27 Sep 2021 10:23:54 AM CEST', 'stderr': '', 'rc': 0, 'start': '2021-09-27 10:23:54.285513', 'end': '2021-09-27 10:23:54.287741', 'delta': '0:00:00.002228', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'date', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['Mon 27 Sep 2021 10:23:54 AM CEST'], 'stderr_lines': [], '_ansible_no_log': False})

In [17]:
# a list of hosts
en.run_command("date", roles = roles["control"])


PLAY [all] **********************************************************************************************************************************************************************************

TASK [date] *********************************************************************************************************************************************************************************
 [started TASK: date on parasilo-23.rennes.grid5000.fr]
changed: [parasilo-23.rennes.grid5000.fr]


CommandResult(host='parasilo-23.rennes.grid5000.fr', task='date', status='OK', payload={'cmd': 'date', 'stdout': 'Mon 27 Sep 2021 10:23:54 AM CEST', 'stderr': '', 'rc': 0, 'start': '2021-09-27 10:23:54.632904', 'end': '2021-09-27 10:23:54.635187', 'delta': '0:00:00.002283', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'date', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['Mon 27 Sep 2021 10:23:54 AM CEST'], 'stderr_lines': [], '_ansible_no_log': False})

In [18]:
# a single host
en.run_command("date", roles=roles["control"][0])


PLAY [all] **********************************************************************************************************************************************************************************

TASK [date] *********************************************************************************************************************************************************************************
 [started TASK: date on parasilo-23.rennes.grid5000.fr]
changed: [parasilo-23.rennes.grid5000.fr]


CommandResult(host='parasilo-23.rennes.grid5000.fr', task='date', status='OK', payload={'cmd': 'date', 'stdout': 'Mon 27 Sep 2021 10:23:55 AM CEST', 'stderr': '', 'rc': 0, 'start': '2021-09-27 10:23:55.000374', 'end': '2021-09-27 10:23:55.002650', 'delta': '0:00:00.002276', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'date', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['Mon 27 Sep 2021 10:23:55 AM CEST'], 'stderr_lines': [], '_ansible_no_log': False})

A `pattern_hosts` can also be supplied. The pattern can be a regexp but [other patterns are possible](
https://docs.ansible.com/ansible/latest/user_guide/intro_patterns.html#common-patterns)

In [19]:
# co* matches all hosts
en.run_command("date", roles=roles, pattern_hosts="co*")

# com* only matches host with `compute` tags
en.run_command("date", roles=roles, pattern_hosts="com*")


PLAY [co*] **********************************************************************************************************************************************************************************

TASK [date] *********************************************************************************************************************************************************************************
 [started TASK: date on parasilo-23.rennes.grid5000.fr]
 [started TASK: date on parasilo-8.rennes.grid5000.fr]
changed: [parasilo-23.rennes.grid5000.fr]
changed: [parasilo-8.rennes.grid5000.fr]

PLAY [com*] *********************************************************************************************************************************************************************************

TASK [date] *********************************************************************************************************************************************************************************
 [started TASK: date on parasilo-8.rennes.

CommandResult(host='parasilo-8.rennes.grid5000.fr', task='date', status='OK', payload={'cmd': 'date', 'stdout': 'Mon 27 Sep 2021 10:23:55 AM CEST', 'stderr': '', 'rc': 0, 'start': '2021-09-27 10:23:55.901765', 'end': '2021-09-27 10:23:55.904136', 'delta': '0:00:00.002371', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'date', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['Mon 27 Sep 2021 10:23:55 AM CEST'], 'stderr_lines': [], '_ansible_no_log': False})

In [20]:
# you can forge some host yourself
# Here we run the command on the frontend: this should work if your SSH parameters are correct
en.run_command("date", roles=en.Host("rennes.grid5000.fr", user=en.g5k_api_utils.get_api_username()))


PLAY [all] **********************************************************************************************************************************************************************************

TASK [date] *********************************************************************************************************************************************************************************
 [started TASK: date on rennes.grid5000.fr]
changed: [rennes.grid5000.fr]


CommandResult(host='rennes.grid5000.fr', task='date', status='OK', payload={'cmd': 'date', 'stdout': 'Mon 27 Sep 2021 10:24:00 AM CEST', 'stderr': '', 'rc': 0, 'start': '2021-09-27 10:24:00.089239', 'end': '2021-09-27 10:24:00.126849', 'delta': '0:00:00.037610', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'date', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['Mon 27 Sep 2021 10:24:00 AM CEST'], 'stderr_lines': [], '_ansible_no_log': False})

### Dealing with failures

By default, failures (command failure, host unreachable) raises on exception: this breaks your execution flow.
Sometime you just want to allow some failures to happen. For this purpose you can add `on_error_continue=True`

In [21]:
en.run_command("non existing command", roles=roles, on_error_continue=True)
print("This is printed, so the execution can continue")


PLAY [all] **********************************************************************************************************************************************************************************

TASK [non existing command] *****************************************************************************************************************************************************************
 [started TASK: non existing command on parasilo-23.rennes.grid5000.fr]
 [started TASK: non existing command on parasilo-8.rennes.grid5000.fr]


fatal: [parasilo-23.rennes.grid5000.fr]: FAILED! => {"changed": true, "cmd": "non existing command", "delta": "0:00:00.001776", "end": "2021-09-27 10:24:01.565074", "msg": "non-zero return code", "rc": 127, "start": "2021-09-27 10:24:01.563298", "stderr": "/bin/sh: 1: non: not found", "stderr_lines": ["/bin/sh: 1: non: not found"], "stdout": "", "stdout_lines": []}
fatal: [parasilo-8.rennes.grid5000.fr]: FAILED! => {"changed": true, "cmd": "non existing command", "delta": "0:00:00.002048", "end": "2021-09-27 10:24:01.612047", "msg": "non-zero return code", "rc": 127, "start": "2021-09-27 10:24:01.609999", "stderr": "/bin/sh: 1: non: not found", "stderr_lines": ["/bin/sh: 1: non: not found"], "stdout": "", "stdout_lines": []}
ERROR:enoslib.api:Failed hosts: [_AnsibleExecutionRecord(host='parasilo-23.rennes.grid5000.fr', status='FAILED', task='non existing command', payload={'cmd': 'non existing command', 'stdout': '', 'stderr': '/bin/sh: 1: non: not found', 'rc': 127, 'start': '2021-09-

This is printed, so the execution can continue


### Remote actions

Tools like Ansible, Puppet, Chef, Terraform ... are shipped with a set of predefined remote actions to ease the administrator life.

Actions like copying file, adding some users, managing packages, making sure a line is absent from a configuration file, managing docker containers ... are first-class citizens actions and brings some nice garantees of correctness and idempotency.

There are 1000+ modules  available:
https://docs.ansible.com/ansible/2.9/modules/list_of_all_modules.html

---

EnOSlib wraps Ansible module and let you use them from Python (without writting any YAML file). You can call any module by using the `actions` context manager:

In the following we install docker (using g5k provided script) and a docker container. We also need to install the python docker binding on the remote machine so that Ansible can interact with the docker daemons on the remote machines. This block of actions is idempotent.


In [22]:
with en.actions(roles=roles) as a:
    # installing the docker daemon
    # prepending with a guard to make the command idempotent
    a.shell("which docker || /grid5000/code/bin/g5k-setup-docker")
    # install the python docker binding on the remote host
    # mandatory by the docker_container module
    a.pip(name="docker", state="present")
    # fire up a container (forward port 80 at the host level)
    a.docker_container(name="myserver", image="nginx", state="started", ports=["80:80"])
    # wait for the connection on the port 80 to be ready
    a.wait_for(port=80, state="started")
    # keep track of the result of each modules
    # not mandatory but nice :)
    results = a.results


PLAY [all] **********************************************************************************************************************************************************************************

TASK [which docker || /grid5000/code/bin/g5k-setup-docker] **********************************************************************************************************************************
 [started TASK: which docker || /grid5000/code/bin/g5k-setup-docker on parasilo-23.rennes.grid5000.fr]
 [started TASK: which docker || /grid5000/code/bin/g5k-setup-docker on parasilo-8.rennes.grid5000.fr]
changed: [parasilo-8.rennes.grid5000.fr]
changed: [parasilo-23.rennes.grid5000.fr]

TASK [pip] **********************************************************************************************************************************************************************************
 [started TASK: pip on parasilo-23.rennes.grid5000.fr]
 [started TASK: pip on parasilo-8.rennes.grid5000.fr]
changed: [parasilo-23.rennes.g

In [23]:
results.filter(task="docker_container")[0]

HostIp,HostPort
0.0.0.0,80
HostIp,HostPort
0.0.0.0,80


### Background actions

Sometime you need to fire a process on some remote machines that needs to survive the remote connection that started it. EnOSlib provides a `keyword` argument for this purpose and can be used when calling modules (when supported).

In [24]:
# synchronous execution, will wait until the end of the shell command
results = en.run_command("for i in $(seq 1 10); do sleep 1; echo toto; done", roles=roles)
results


PLAY [all] **********************************************************************************************************************************************************************************

TASK [for i in $(seq 1 10); do sleep 1; echo toto; done] ************************************************************************************************************************************
 [started TASK: for i in $(seq 1 10); do sleep 1; echo toto; done on parasilo-23.rennes.grid5000.fr]
 [started TASK: for i in $(seq 1 10); do sleep 1; echo toto; done on parasilo-8.rennes.grid5000.fr]
changed: [parasilo-23.rennes.grid5000.fr]
changed: [parasilo-8.rennes.grid5000.fr]


CommandResult(host='parasilo-23.rennes.grid5000.fr', task='for i in $(seq 1 10); do sleep 1; echo toto; done', status='OK', payload={'cmd': 'for i in $(seq 1 10); do sleep 1; echo toto; done', 'stdout': 'toto\ntoto\ntoto\ntoto\ntoto\ntoto\ntoto\ntoto\ntoto\ntoto', 'stderr': '', 'rc': 0, 'start': '2021-09-27 10:24:47.480422', 'end': '2021-09-27 10:24:57.493828', 'delta': '0:00:10.013406', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'for i in $(seq 1 10); do sleep 1; echo toto; done', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['toto', 'toto', 'toto', 'toto', 'toto', 'toto', 'toto', 'toto', 'toto', 'toto'], 'stderr_lines': [], '_ansible_no_log': False})
CommandResult(host='parasilo-8.rennes.grid5000.fr', task='for i in $(seq 1 10); do sleep 1; echo toto; done', status='OK', payload={'cmd': 'for i in $(seq 1 10); do sleep 1; echo toto; done', 'stdout': 'toto\ntoto\ntoto\ntoto\ntoto\ntoto\ntoto\ntoto\ntoto\ntoto', 'stderr': '', 'rc': 0, 'start': '2021-09-27 10:24:47.494375', 'end': '2021-09-27 10:24:57.507814', 'delta': '0:00:10.013439', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'for i in $(seq 1 10); do sleep 1; echo toto; done', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['toto', 'toto', 'toto', 'toto', 'toto', 'toto', 'toto', 'toto', 'toto', 'toto'], 'stderr_lines': [], '_ansible_no_log': False})

In [25]:
# The remote command will be daemonize on the remote machines
results = en.run_command("for i in $(seq 1 10); do sleep 1; echo toto; done", roles=roles, background=True)
results


PLAY [all] **********************************************************************************************************************************************************************************

TASK [for i in $(seq 1 10); do sleep 1; echo toto; done] ************************************************************************************************************************************
 [started TASK: for i in $(seq 1 10); do sleep 1; echo toto; done on parasilo-23.rennes.grid5000.fr]
 [started TASK: for i in $(seq 1 10); do sleep 1; echo toto; done on parasilo-8.rennes.grid5000.fr]
changed: [parasilo-23.rennes.grid5000.fr]
changed: [parasilo-8.rennes.grid5000.fr]


AsyncCommandResult(host='parasilo-23.rennes.grid5000.fr', task='for i in $(seq 1 10); do sleep 1; echo toto; done', status='OK', payload={'started': 1, 'finished': 0, 'ansible_job_id': '809819000949.11356', 'results_file': '/root/.ansible_async/809819000949.11356', 'changed': True, '_ansible_no_log': False})
AsyncCommandResult(host='parasilo-8.rennes.grid5000.fr', task='for i in $(seq 1 10); do sleep 1; echo toto; done', status='OK', payload={'started': 1, 'finished': 0, 'ansible_job_id': '642805157253.24496', 'results_file': '/root/.ansible_async/642805157253.24496', 'changed': True, '_ansible_no_log': False})

In [26]:
# you can get back the status of the daemonized process by reading the remote results_file
# but we need to wait the completion, so forcing a sleep here (one could poll the status)
import time
time.sleep(15)
h  = roles["control"][0]
result_file = results.filter(host=h.alias)[0].results_file
cat_result = en.run_command(f"cat {result_file}",roles=h)
cat_result


PLAY [all] **********************************************************************************************************************************************************************************

TASK [cat /root/.ansible_async/809819000949.11356] ******************************************************************************************************************************************
 [started TASK: cat /root/.ansible_async/809819000949.11356 on parasilo-23.rennes.grid5000.fr]
changed: [parasilo-23.rennes.grid5000.fr]


CommandResult(host='parasilo-23.rennes.grid5000.fr', task='cat /root/.ansible_async/809819000949.11356', status='OK', payload={'cmd': 'cat /root/.ansible_async/809819000949.11356', 'stdout': '{"cmd": "for i in $(seq 1 10); do sleep 1; echo toto; done", "stdout": "toto\\ntoto\\ntoto\\ntoto\\ntoto\\ntoto\\ntoto\\ntoto\\ntoto\\ntoto", "stderr": "", "rc": 0, "start": "2021-09-27 10:24:59.713197", "end": "2021-09-27 10:25:09.722981", "delta": "0:00:10.009784", "changed": true, "invocation": {"module_args": {"_raw_params": "for i in $(seq 1 10); do sleep 1; echo toto; done", "_uses_shell": true, "warn": true, "stdin_add_newline": true, "strip_empty_ends": true, "argv": null, "chdir": null, "executable": null, "creates": null, "removes": null, "stdin": null}}}', 'stderr': '', 'rc': 0, 'start': '2021-09-27 10:25:15.079804', 'end': '2021-09-27 10:25:15.082314', 'delta': '0:00:00.002510', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'cat /root/.ansible_async/809819000949.11356', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['{"cmd": "for i in $(seq 1 10); do sleep 1; echo toto; done", "stdout": "toto\\ntoto\\ntoto\\ntoto\\ntoto\\ntoto\\ntoto\\ntoto\\ntoto\\ntoto", "stderr": "", "rc": 0, "start": "2021-09-27 10:24:59.713197", "end": "2021-09-27 10:25:09.722981", "delta": "0:00:10.009784", "changed": true, "invocation": {"module_args": {"_raw_params": "for i in $(seq 1 10); do sleep 1; echo toto; done", "_uses_shell": true, "warn": true, "stdin_add_newline": true, "strip_empty_ends": true, "argv": null, "chdir": null, "executable": null, "creates": null, "removes": null, "stdin": null}}}'], 'stderr_lines': [], '_ansible_no_log': False})

In [27]:
# the result_file content is json encoded so decoding it
import json
print(json.loads(cat_result[0].stdout)["stdout"])

toto
toto
toto
toto
toto
toto
toto
toto
toto
toto


## Using variables

### Same variable value for everyone

Nothing surprising here, you can use regular python interpolation (e.g a `f-string`).
String are interpolated by the interpreter before being manipulated.

In [28]:
host_to_ping = roles["control"][0].alias
host_to_ping

results = en.run_command(f"ping -c 5 {host_to_ping}", roles=roles)
results


PLAY [all] **********************************************************************************************************************************************************************************

TASK [ping -c 5 parasilo-23.rennes.grid5000.fr] *********************************************************************************************************************************************
 [started TASK: ping -c 5 parasilo-23.rennes.grid5000.fr on parasilo-23.rennes.grid5000.fr]
 [started TASK: ping -c 5 parasilo-23.rennes.grid5000.fr on parasilo-8.rennes.grid5000.fr]
changed: [parasilo-23.rennes.grid5000.fr]
changed: [parasilo-8.rennes.grid5000.fr]


CommandResult(host='parasilo-23.rennes.grid5000.fr', task='ping -c 5 parasilo-23.rennes.grid5000.fr', status='OK', payload={'cmd': 'ping -c 5 parasilo-23.rennes.grid5000.fr', 'stdout': 'PING parasilo-23.rennes.grid5000.fr (172.16.97.23) 56(84) bytes of data.\n64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=1 ttl=64 time=0.021 ms\n64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=2 ttl=64 time=0.024 ms\n64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=3 ttl=64 time=0.012 ms\n64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=4 ttl=64 time=0.015 ms\n64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=5 ttl=64 time=0.031 ms\n\n--- parasilo-23.rennes.grid5000.fr ping statistics ---\n5 packets transmitted, 5 received, 0% packet loss, time 85ms\nrtt min/avg/max/mdev = 0.012/0.020/0.031/0.008 ms', 'stderr': '', 'rc': 0, 'start': '2021-09-27 10:25:15.414730', 'end': '2021-09-27 10:25:19.505219', 'delta': '0:00:04.090489', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'ping -c 5 parasilo-23.rennes.grid5000.fr', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['PING parasilo-23.rennes.grid5000.fr (172.16.97.23) 56(84) bytes of data.', '64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=1 ttl=64 time=0.021 ms', '64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=2 ttl=64 time=0.024 ms', '64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=3 ttl=64 time=0.012 ms', '64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=4 ttl=64 time=0.015 ms', '64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=5 ttl=64 time=0.031 ms', '', '--- parasilo-23.rennes.grid5000.fr ping statistics ---', '5 packets transmitted, 5 received, 0% packet loss, time 85ms', 'rtt min/avg/max/mdev = 0.012/0.020/0.031/0.008 ms'], 'stderr_lines': [], '_ansible_no_log': False})
CommandResult(host='parasilo-8.rennes.grid5000.fr', task='ping -c 5 parasilo-23.rennes.grid5000.fr', status='OK', payload={'cmd': 'ping -c 5 parasilo-23.rennes.grid5000.fr', 'stdout': 'PING parasilo-23.rennes.grid5000.fr (172.16.97.23) 56(84) bytes of data.\n64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=1 ttl=64 time=0.199 ms\n64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=2 ttl=64 time=0.119 ms\n64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=3 ttl=64 time=0.113 ms\n64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=4 ttl=64 time=0.163 ms\n64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=5 ttl=64 time=0.207 ms\n\n--- parasilo-23.rennes.grid5000.fr ping statistics ---\n5 packets transmitted, 5 received, 0% packet loss, time 81ms\nrtt min/avg/max/mdev = 0.113/0.160/0.207/0.039 ms', 'stderr': '', 'rc': 0, 'start': '2021-09-27 10:25:15.514819', 'end': '2021-09-27 10:25:19.602286', 'delta': '0:00:04.087467', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'ping -c 5 parasilo-23.rennes.grid5000.fr', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['PING parasilo-23.rennes.grid5000.fr (172.16.97.23) 56(84) bytes of data.', '64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=1 ttl=64 time=0.199 ms', '64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=2 ttl=64 time=0.119 ms', '64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=3 ttl=64 time=0.113 ms', '64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=4 ttl=64 time=0.163 ms', '64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=5 ttl=6

In [29]:
[(r.host, r.stdout) for r in results]

[('parasilo-23.rennes.grid5000.fr',
  'PING parasilo-23.rennes.grid5000.fr (172.16.97.23) 56(84) bytes of data.\n64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=1 ttl=64 time=0.021 ms\n64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=2 ttl=64 time=0.024 ms\n64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=3 ttl=64 time=0.012 ms\n64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=4 ttl=64 time=0.015 ms\n64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=5 ttl=64 time=0.031 ms\n\n--- parasilo-23.rennes.grid5000.fr ping statistics ---\n5 packets transmitted, 5 received, 0% packet loss, time 85ms\nrtt min/avg/max/mdev = 0.012/0.020/0.031/0.008 ms'),
 ('parasilo-8.rennes.grid5000.fr',
  'PING parasilo-23.rennes.grid5000.fr (172.16.97.23) 56(84) bytes of data.\n64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=1 ttl=64 time=0.199 ms\n64 bytes from parasilo-23.rennes.grid5

### Using templates / Ansible variables

There's an alternative way to pass a variable to a task: using `extra_vars`.
The difference with the previous case (python interpreted variables) is the fact that the variable is interpolated right before execution happens on the remote node.
One could imagine the the value is broadcasted to all nodes and replaced right before the execution.

To indicate that we want to use this kind of variables, we need to pass its value using the `extra_vars` dictionnary and use a template (`{{ ... }}`) in the task description.

In [30]:
host_to_ping = roles["control"][0].alias
host_to_ping

results = en.run_command("ping -c 5 {{ my_template_variable }}", roles=roles, extra_vars=dict(my_template_variable=host_to_ping))
results


PLAY [all] **********************************************************************************************************************************************************************************

TASK [ping -c 5 parasilo-23.rennes.grid5000.fr] *********************************************************************************************************************************************
 [started TASK: ping -c 5 {{ my_template_variable }} on parasilo-23.rennes.grid5000.fr]
 [started TASK: ping -c 5 {{ my_template_variable }} on parasilo-8.rennes.grid5000.fr]
changed: [parasilo-23.rennes.grid5000.fr]
changed: [parasilo-8.rennes.grid5000.fr]


CommandResult(host='parasilo-23.rennes.grid5000.fr', task='ping -c 5 parasilo-23.rennes.grid5000.fr', status='OK', payload={'cmd': 'ping -c 5 parasilo-23.rennes.grid5000.fr', 'stdout': 'PING parasilo-23.rennes.grid5000.fr (172.16.97.23) 56(84) bytes of data.\n64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=1 ttl=64 time=0.018 ms\n64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=2 ttl=64 time=0.031 ms\n64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=3 ttl=64 time=0.017 ms\n64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=4 ttl=64 time=0.030 ms\n64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=5 ttl=64 time=0.031 ms\n\n--- parasilo-23.rennes.grid5000.fr ping statistics ---\n5 packets transmitted, 5 received, 0% packet loss, time 91ms\nrtt min/avg/max/mdev = 0.017/0.025/0.031/0.007 ms', 'stderr': '', 'rc': 0, 'start': '2021-09-27 10:25:20.053534', 'end': '2021-09-27 10:25:24.145304', 'delta': '0:00:04.091770', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'ping -c 5 parasilo-23.rennes.grid5000.fr', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['PING parasilo-23.rennes.grid5000.fr (172.16.97.23) 56(84) bytes of data.', '64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=1 ttl=64 time=0.018 ms', '64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=2 ttl=64 time=0.031 ms', '64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=3 ttl=64 time=0.017 ms', '64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=4 ttl=64 time=0.030 ms', '64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=5 ttl=64 time=0.031 ms', '', '--- parasilo-23.rennes.grid5000.fr ping statistics ---', '5 packets transmitted, 5 received, 0% packet loss, time 91ms', 'rtt min/avg/max/mdev = 0.017/0.025/0.031/0.007 ms'], 'stderr_lines': [], '_ansible_no_log': False})
CommandResult(host='parasilo-8.rennes.grid5000.fr', task='ping -c 5 parasilo-23.rennes.grid5000.fr', status='OK', payload={'cmd': 'ping -c 5 parasilo-23.rennes.grid5000.fr', 'stdout': 'PING parasilo-23.rennes.grid5000.fr (172.16.97.23) 56(84) bytes of data.\n64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=1 ttl=64 time=0.139 ms\n64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=2 ttl=64 time=0.183 ms\n64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=3 ttl=64 time=0.157 ms\n64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=4 ttl=64 time=0.175 ms\n64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=5 ttl=64 time=0.181 ms\n\n--- parasilo-23.rennes.grid5000.fr ping statistics ---\n5 packets transmitted, 5 received, 0% packet loss, time 81ms\nrtt min/avg/max/mdev = 0.139/0.167/0.183/0.016 ms', 'stderr': '', 'rc': 0, 'start': '2021-09-27 10:25:20.096548', 'end': '2021-09-27 10:25:24.178280', 'delta': '0:00:04.081732', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'ping -c 5 parasilo-23.rennes.grid5000.fr', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['PING parasilo-23.rennes.grid5000.fr (172.16.97.23) 56(84) bytes of data.', '64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=1 ttl=64 time=0.139 ms', '64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=2 ttl=64 time=0.183 ms', '64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=3 ttl=64 time=0.157 ms', '64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=4 ttl=64 time=0.175 ms', '64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=5 ttl=6

### Host specific variables

In the above, we've seen how a common value can be broadcasted to all remote nodes.  What if we want host specific value ?

For instance in our case we'd like `host 1` to ping `host 2` and `host 2` to ping `host 1`. That make the `host_to_ping` variable host-specific.

For this purpose you can use the `extra` attribute of the `Host` objects and use a template as before.

In [31]:
control_host = roles["control"][0]
compute_host = roles["compute"][0]
control_host.extra.update(host_to_ping=compute_host.address)
compute_host.extra.update(host_to_ping=control_host.address)
control_host

ip
::1/128
127.0.0.1/8
ip
fe80::eef4:bbff:fed1:e60/64
172.16.97.23/20


> Note that the `extra` attribute is mutable :(

In [32]:
results = en.run_command("ping -c 5 {{ host_to_ping }}", roles=roles)
results


PLAY [all] **********************************************************************************************************************************************************************************

TASK [ping -c 5 parasilo-8.rennes.grid5000.fr] **********************************************************************************************************************************************
 [started TASK: ping -c 5 {{ host_to_ping }} on parasilo-23.rennes.grid5000.fr]
 [started TASK: ping -c 5 {{ host_to_ping }} on parasilo-8.rennes.grid5000.fr]
changed: [parasilo-23.rennes.grid5000.fr]
changed: [parasilo-8.rennes.grid5000.fr]


CommandResult(host='parasilo-23.rennes.grid5000.fr', task='ping -c 5 parasilo-8.rennes.grid5000.fr', status='OK', payload={'cmd': 'ping -c 5 parasilo-8.rennes.grid5000.fr', 'stdout': 'PING parasilo-8.rennes.grid5000.fr (172.16.97.8) 56(84) bytes of data.\n64 bytes from parasilo-8.rennes.grid5000.fr (172.16.97.8): icmp_seq=1 ttl=64 time=0.160 ms\n64 bytes from parasilo-8.rennes.grid5000.fr (172.16.97.8): icmp_seq=2 ttl=64 time=0.174 ms\n64 bytes from parasilo-8.rennes.grid5000.fr (172.16.97.8): icmp_seq=3 ttl=64 time=0.187 ms\n64 bytes from parasilo-8.rennes.grid5000.fr (172.16.97.8): icmp_seq=4 ttl=64 time=0.203 ms\n64 bytes from parasilo-8.rennes.grid5000.fr (172.16.97.8): icmp_seq=5 ttl=64 time=0.201 ms\n\n--- parasilo-8.rennes.grid5000.fr ping statistics ---\n5 packets transmitted, 5 received, 0% packet loss, time 85ms\nrtt min/avg/max/mdev = 0.160/0.185/0.203/0.016 ms', 'stderr': '', 'rc': 0, 'start': '2021-09-27 10:25:24.603935', 'end': '2021-09-27 10:25:28.689355', 'delta': '0:00:04.085420', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'ping -c 5 parasilo-8.rennes.grid5000.fr', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['PING parasilo-8.rennes.grid5000.fr (172.16.97.8) 56(84) bytes of data.', '64 bytes from parasilo-8.rennes.grid5000.fr (172.16.97.8): icmp_seq=1 ttl=64 time=0.160 ms', '64 bytes from parasilo-8.rennes.grid5000.fr (172.16.97.8): icmp_seq=2 ttl=64 time=0.174 ms', '64 bytes from parasilo-8.rennes.grid5000.fr (172.16.97.8): icmp_seq=3 ttl=64 time=0.187 ms', '64 bytes from parasilo-8.rennes.grid5000.fr (172.16.97.8): icmp_seq=4 ttl=64 time=0.203 ms', '64 bytes from parasilo-8.rennes.grid5000.fr (172.16.97.8): icmp_seq=5 ttl=64 time=0.201 ms', '', '--- parasilo-8.rennes.grid5000.fr ping statistics ---', '5 packets transmitted, 5 received, 0% packet loss, time 85ms', 'rtt min/avg/max/mdev = 0.160/0.185/0.203/0.016 ms'], 'stderr_lines': [], '_ansible_no_log': False})
CommandResult(host='parasilo-8.rennes.grid5000.fr', task='ping -c 5 parasilo-23.rennes.grid5000.fr', status='OK', payload={'cmd': 'ping -c 5 parasilo-23.rennes.grid5000.fr', 'stdout': 'PING parasilo-23.rennes.grid5000.fr (172.16.97.23) 56(84) bytes of data.\n64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=1 ttl=64 time=0.155 ms\n64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=2 ttl=64 time=0.182 ms\n64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=3 ttl=64 time=0.173 ms\n64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=4 ttl=64 time=0.142 ms\n64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=5 ttl=64 time=0.160 ms\n\n--- parasilo-23.rennes.grid5000.fr ping statistics ---\n5 packets transmitted, 5 received, 0% packet loss, time 88ms\nrtt min/avg/max/mdev = 0.142/0.162/0.182/0.018 ms', 'stderr': '', 'rc': 0, 'start': '2021-09-27 10:25:24.633940', 'end': '2021-09-27 10:25:28.722100', 'delta': '0:00:04.088160', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'ping -c 5 parasilo-23.rennes.grid5000.fr', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['PING parasilo-23.rennes.grid5000.fr (172.16.97.23) 56(84) bytes of data.', '64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=1 ttl=64 time=0.155 ms', '64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=2 ttl=64 time=0.182 ms', '64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=3 ttl=64 time=0.173 ms', '64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=4 ttl=64 time=0.142 ms', '64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=5 ttl=64 time=0.160 ms', '', '--- pa

In [33]:
[(r.host, r.stdout) for r in results]

[('parasilo-23.rennes.grid5000.fr',
  'PING parasilo-8.rennes.grid5000.fr (172.16.97.8) 56(84) bytes of data.\n64 bytes from parasilo-8.rennes.grid5000.fr (172.16.97.8): icmp_seq=1 ttl=64 time=0.160 ms\n64 bytes from parasilo-8.rennes.grid5000.fr (172.16.97.8): icmp_seq=2 ttl=64 time=0.174 ms\n64 bytes from parasilo-8.rennes.grid5000.fr (172.16.97.8): icmp_seq=3 ttl=64 time=0.187 ms\n64 bytes from parasilo-8.rennes.grid5000.fr (172.16.97.8): icmp_seq=4 ttl=64 time=0.203 ms\n64 bytes from parasilo-8.rennes.grid5000.fr (172.16.97.8): icmp_seq=5 ttl=64 time=0.201 ms\n\n--- parasilo-8.rennes.grid5000.fr ping statistics ---\n5 packets transmitted, 5 received, 0% packet loss, time 85ms\nrtt min/avg/max/mdev = 0.160/0.185/0.203/0.016 ms'),
 ('parasilo-8.rennes.grid5000.fr',
  'PING parasilo-23.rennes.grid5000.fr (172.16.97.23) 56(84) bytes of data.\n64 bytes from parasilo-23.rennes.grid5000.fr (172.16.97.23): icmp_seq=1 ttl=64 time=0.155 ms\n64 bytes from parasilo-23.rennes.grid5000.fr (172.1

## Cleaning

In [34]:
provider.destroy()

INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from grenoble
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from lille
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from luxembourg
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from lyon
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from nancy
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from nantes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from rennes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading 1820191 from rennes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from sophia
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Killing the job (rennes, 1820191)
